In [1]:
############################################################################################################################
## Script: AGOLSQLSync.py
## Author: Jeremy Mullins, Derek Morgan
## Date:
##
## Description:
##        This script is the test script for syncing data between ArcGIS Online
##        and an MS SQL Server database.
##
## Required prerequisites:
##        - ArcGIS API for Python
##            (https://developers.arcgis.com/python/)
##
##        - pyodbc Module
##            (https://github.com/mkleehammer/pyodbc)
##                to install after download, open Python command
##                prompt and type the following:
##                    -- conda install pyodbc  --
##
##        - MS ODBC Driver for SQL Server (v17)
##            (https://docs.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-2017)
##
############################################################################################################################

In [2]:
# import all necessary modules
from arcgis.gis import GIS
from arcgis import geometry
from copy import deepcopy
import pyodbc as sql
import pandas as pd
import configparser
import zipfile
import os

In [8]:
# read config file
config = configparser.ConfigParser()
config.read('config_jm.ini')

# assign config variables
agolURL = config['AGOL']['URL']
agolUSER = config['AGOL']['USER']
agolPW = config['AGOL']['PW']
sqlDRVR = config['SQL']['SQLDRVR']
sqlSERV = config['SQL']['SERVER']
sqlDB = config['SQL']['DB']
sqlUSER = config['SQL']['USER']
sqlPW = config['SQL']['PW']
iD = config['SCRIPT']['itemID']
csvNAME = config['SCRIPT']['csvTITLE']
csvLOC = config['SCRIPT']['csvLOC']
zipLOC = config['SCRIPT']['zipLOC']
csvDOC = config['SCRIPT']['csvDOC']
newTEMP1 = config['SCRIPT']['newTEMP1']
newTEMP2 = config['SCRIPT']['newTEMP2']
csv2TEMP = config['SCRIPT']['csv2TEMP']
newINS1 = config['SCRIPT']['newINS1']
newINS2 = config['SCRIPT']['newINS2']
SQL2CSV = config['SCRIPT']['SQL2CSV']
sql2CSVout = config['SCRIPT']['sql2CSVout']
delTEMP1 = config['SCRIPT']['delTEMP1']
delTEMP2 = config['SCRIPT']['delTEMP2']

In [9]:
# sign into AGOL acct
gis = GIS(agolURL,agolUSER,agolPW)

# get feature layer in question
featureLayer = gis.content.get(iD)

In [10]:
# export feature layer as CSV (ZIP file)
output_file = featureLayer.export(title=csvNAME,export_format="CSV")
output_file.download(csvLOC)

'W:\\\\src\\\\Data\\uwfBuildings.zip'

In [11]:
# unzip downloaded ZIP folder containing feature layer as CSV
zip_ref = zipfile.ZipFile(zipLOC,'r')
zip_ref.extractall(csvLOC)
zip_ref.close()

# delete ZIP folder on disk and CSV collection file in AGOL
os.remove(zipLOC)
output_file.delete()

True

In [12]:
# read CSV using panda; replace NaNs with '000'
df = pd.read_csv(csvDOC)
df.fillna('000',inplace=True)

In [16]:
df

,FID,BUILDINGID,Descriptio,Classrooms,TypeCode,email,x,y
0,1,76,COLLEGE OF BUSINESS,No,1,,-87.215501,30.549540
1,2,46,HOUSING MAINTENANCE,No,5,,-87.218518,30.553227
2,3,53,COB RAYMOND M. HAAS CENTER,No,1,,-87.215550,30.550373
3,4,T14,,,0,,-87.204129,30.541286
4,5,219,,No,0,,-87.203419,30.540187
5,6,89,MARGARET J SMITH ARCHAEOLOGY,No,1,,-87.217682,30.542246
6,7,50,HUMA & SOC SCIENCES OFFICES,No,1,,-87.216707,30.551490
7,8,99,,No,2,,-87.214874,30.543081
8,9,52,CLASSROOMS/OFFICES,Yes,1,,-87.215864,30.550477
9,10,46,HOUSING MAINTENANCE,No,5,,-87.217279,30.554813


In [14]:
#Connect to SQL db and assign cursor
conn = sql.connect('Driver='+sqlDRVR+';'
                      'Server='+sqlSERV+';'
                      'Database='+sqlDB+';'
                      'trusted_connection=yes;'
                      'UID='+sqlUSER+';'
                      'PWD='+sqlPW+';')
cursor = conn.cursor()

# create temporary tables in SQL db
cursor.execute(newTEMP1)
cursor.commit()

cursor.execute(newTEMP2)
cursor.commit()

In [17]:
# insert CSV into temporary table
for index,row in df.iterrows():
    cursor.execute(csv2TEMP,row['FID'],
                   row['BUILDINGID'],
                   row['Descriptio'],
                   row['Classrooms'],
                   row['TypeCode'],
                   row['email'],
                   row['x'],
                   row['y']
                  )
cursor.commit()

In [18]:
# compare tables and look for non-duplicated GlobalIDs
cursor.execute(newINS1)
cursor.commit()

In [19]:
# compare tables and look for any updated records in prod table
cursor.execute(newINS2)
cursor.commit()

In [20]:
# export SQL data to CSV
outCSVscript = SQL2CSV

df2 = pd.read_sql_query(outCSVscript,conn)

df2.to_csv(sql2CSVout)

# https://stackoverflow.com/questions/
#     18977387/how-to-export-sql-server-result-to-excel-in-python
# https://datatofish.com/export-dataframe-to-csv/

In [21]:
# read the SQL update CSV
csv2 = sql2CSVout
updates_df_2 = pd.read_csv(csv2)

# query all features in target AGOL feature layer
fl = featureLayer.layers[0]
flquery = fl.query()

# determine which features overlap between update CSV and AGOL feature layer using an INNER join
overlap_rows = pd.merge(left = flquery.sdf, right = updates_df_2, how = 'inner', on = 'FID')

# create list containing corrected features (ftbu)
updatefeatures = []
all_features = flquery.features

In [23]:
# for loop to prepare updated geometries and attributes for each of the updated features
# geometry module used to project coordinates form geographic to projected coordinate system
for fid in overlap_rows['FID']:
    # get the feature to be updated
    original_feature = [f for f in all_features if f.attributes['FID'] == fid][0]
    ftbu = deepcopy(original_feature)
    
    # get the matching row from csv
    matching_row = updates_df_2.where(updates_df_2.FID == fid).dropna()
    
    # get geometries in the destination coordinate system
    input_geometry = {'y':float(matching_row['Y']),
                      'x':float(matching_row['X'])}
    output_geometry = geometry.project(geometries = [input_geometry],
                                       in_sr = 4326,
                                       out_sr = flquery.spatial_reference['latestWkid'],
                                       gis = gis)
    # assign the updated values
    ftbu.geometry = output_geometry[0]
    ftbu.attributes['BUILDINGID'] = matching_row['BuildingID'].values[0]
    ftbu.attributes['Classrooms'] = matching_row['Classrooms'].values[0]
    ftbu.attributes['Descriptio'] = matching_row['Description'].values[0]
    ftbu.attributes['TypeCode'] = matching_row['TypeCode'].values[0]
    ftbu.attributes['FID'] = int(matching_row['FID'])
    ftbu.attributes['Email'] = matching_row['Email'].values[0]
    
    # add this to the list of features to be updated
    updatefeatures.append(ftbu)

In [24]:
updatefeatures

[{"geometry": {"x": -9708785.2076, "y": 3574385.459000003}, "attributes": {"FID": 1, "BUILDINGID": "76", "Descriptio": "COLLEGE OF BUSINESS", "Classrooms": "No", "TypeCode": 1.0, "email": " ", "SHAPE": {"x": -9708785.2076, "y": 3574385.458999999, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "Email": "000"}},
 {"geometry": {"x": -9709120.994, "y": 3574862.0340000084}, "attributes": {"FID": 2, "BUILDINGID": "46", "Descriptio": "HOUSING MAINTENANCE", "Classrooms": "No", "TypeCode": 5.0, "email": " ", "SHAPE": {"x": -9709120.994, "y": 3574862.034000002, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "Email": "000"}},
 {"geometry": {"x": -9708790.588499995, "y": 3574493.1631000014}, "attributes": {"FID": 3, "BUILDINGID": "53", "Descriptio": "COB RAYMOND M. HAAS CENTER", "Classrooms": "No", "TypeCode": 1.0, "email": " ", "SHAPE": {"x": -9708790.5885, "y": 3574493.1630999967, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "Email": "000"}},
 {"geometry": {

In [25]:
# call the edit_features() method of FeatureLayer object and pass features to the updates parameter
fl.edit_features(updates = updatefeatures)
raise ValueError()
# https://developers.arcgis.com/python/sample-notebooks/updating-features-in-a-feature-layer/

Cannot perform operation. Invalid operation parameters.
'updates' parameter is invalid
An item with the same key has already been added.


RuntimeError: Cannot perform operation. Invalid operation parameters.
'updates' parameter is invalid
An item with the same key has already been added.
(Error Code: 400)

In [26]:
# delete temporary table
cursor.execute(delTEMP1)
cursor.commit()

cursor.execute(delTEMP2)
cursor.commit()

In [27]:
# close and delete cursor; close SQL db connection
cursor.close()
del cursor
conn.close()

In [28]:
# remove all CSVs
os.remove(csvDOC)
os.remove(sql2CSVout)